In [1]:
'''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''

'\nAuthor information:\nJoey R. Muffoletto\nUniversity of Texas at Austin\nAutonomous Systems Group\njrmuff@utexas.edu\n'

In [2]:
'''
imports
'''
import sys
import os
sys.path.insert(0, os.path.abspath('C:\\Users\\Joey\\PycharmProjects\\Automata-Testing'))
import copy
import optimization_functions
import graph_manager
import graph_test_methods
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz 2.44.1/bin'

c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [32]:
'''
Globals/System variables
'''

NUM_TOWERS = 5 
RUNS_PER_DATA_POINT = 1
LINES_PER_SYSTEM = 1000 # how many lines of data to parse for tower matches
BATCH_SIZES = [200, 400] #how many time steps to parse for tower matches

# batch size stuff -> show how batch size effects optimality
# change to time increments (batch size based on time increment)
# take the sum of the cost vs. batch size, keep synthesis time around (average time per batch)
# should show that as batch size goes down, cost goes up, but synthesis time goes down

#DESTINATIONS = ['K'+ str(i) for i in range(1, NUM_TOWERS+1)] #randomize this?
# DESTINATIONS = dict()
# for i in range(1, NUM_TOWERS+1):
#     DESTINATIONS['K' + str(i)] = []

# DATA indicies
EVENT_TIME_INDEX = 0
ORIGIN_INDEX = 1
DESTINATION_INDEX = 2
DEPARTURE_TIME_INDEX = 3
NUM_PASSENGERS_INDEX = 4


In [34]:
'''
Read NASA data
'''
import csv
import copy

DESTINATIONS = {'K4' : [], 'K6' : [], 'K10' : [], 'K11' : [], 'K14': []}

# print(DESTINATIONS)

data = open('data/scn_UAM_testNewVT.trp', 'r')


reader = csv.reader(data)
next(reader) #skip headers
next(reader) #skip units

# structure = destinations_per_batch_size_per_batch[batch_size_index][batch_index] = destination_dict
destinations_per_batch_size_per_batch = [[copy.deepcopy(DESTINATIONS)] for i in range(len(BATCH_SIZES))]
# print(type(destinations_per_batch_size_per_batch[0]))

# print(destinations_per_batch_size_per_batch)
time_stamp_per_batch_size = [0 for i in range(len(BATCH_SIZES))]
previous_time = 21633 # first time in file
for row in reader:    
    for index, time_stamp in enumerate(time_stamp_per_batch_size):
        if time_stamp >= BATCH_SIZES[index]:
            destinations_per_batch_size_per_batch[index].append(copy.deepcopy(DESTINATIONS))
            time_stamp_per_batch_size[index] = 0

#     for index, batch_size in enumerate(BATCH_SIZES):
#         if time_stamp % batch_size == 0:
#             last_item_index = len(destinations_per_batch_size_per_batch[index]) - 1 
    

    eventTime, origin, destination, desiredDepartureTime, numPassengers = row
    time_interval = float(eventTime) - previous_time
    previous_time = float(eventTime)
    
    for index in range(len(time_stamp_per_batch_size)):
        time_stamp_per_batch_size[index] += time_interval
        
    if destination in DESTINATIONS.keys():
        for index in range(len(destinations_per_batch_size_per_batch)):
            last_item_index = len(destinations_per_batch_size_per_batch[index]) - 1 
            destinations_per_batch_size_per_batch[index][last_item_index][destination].append(desiredDepartureTime)


print(len(destinations_per_batch_size_per_batch[0]))
print("")
print(len(destinations_per_batch_size_per_batch[1]))
for row in destinations_per_batch_size_per_batch[1]:
    for key in row.keys():
        print(len(row[key]))

54

27
0
0
0
0
0
1
6
0
0
0
2
5
3
1
0
0
3
0
1
1
0
6
1
0
2
2
5
6
0
0
2
4
1
2
1
4
5
0
0
1
11
9
4
3
3
2
5
1
1
0
3
7
2
0
0
3
4
2
0
1
1
5
4
2
5
5
8
1
0
4
17
2
3
0
3
16
10
1
0
1
12
7
0
2
0
4
7
0
2
1
5
0
1
1
2
5
0
0
1
2
2
1
2
1
1
1
1
1
3
2
1
0
0
2
2
2
4
0
2
1
1
0
0
2
6
4
1
4
0
6
3
3
7
3
14


In [18]:
'''
Create a system per interval

current choices:
    - requests have no preference
    - one port per tower with 3 capacity
    - request time is equal to index + 1, capped at 5. for 7 requests = [1,2,3,4,5,5,5]
    - request vec size is capped to 7
'''


system_per_batch_per_interval = [[] for i in range(len(BATCH_SIZES))]

TIME_CAP = 3
REQ_CAP = 9
for index, system_per_interval in enumerate(system_per_batch_per_interval):
    print("\n\nnew batch size " + str(BATCH_SIZES[index]))
    for count, row in enumerate(destinations_per_batch_size_per_batch[index]):
        print("new system " + str(count))
        system = []
        for key in row.keys():
        #graph_manager.return_tower_specific(port_dict, req_per_step, request_vec, time_req)
            port_dict = {'0' : 3}
            req_per_step = 1
            request_vec = []
            for index, i in enumerate(row[key]):
                if index >= REQ_CAP:
                    break
                request_vec.append('no_pref')
            
            time_vec = []
            for i in range(1, len(request_vec)+1):
                if(i > TIME_CAP):
                    time_vec.append(TIME_CAP)
                else:
                    time_vec.append(i)

            assert(len(request_vec) == len(time_vec))
#             print("new tower")
            tower = graph_manager.return_tower_specific(port_dict, req_per_step, request_vec, time_vec)
            system.append(tower)
        assert(len(system) == len(row.keys()))
        system_per_interval.append(system)
        print(system)



new batch size 500
new system 0
[<reworked_graph.ReworkedGraph object at 0x000001EA194F11D0>, <reworked_graph.ReworkedGraph object at 0x000001EA194F1630>, <reworked_graph.ReworkedGraph object at 0x000001EA194F1048>, <reworked_graph.ReworkedGraph object at 0x000001EA194F1438>, <reworked_graph.ReworkedGraph object at 0x000001EA194F1588>]
new system 1
[<reworked_graph.ReworkedGraph object at 0x000001EA194F15C0>, <reworked_graph.ReworkedGraph object at 0x000001EA194F1D68>, <reworked_graph.ReworkedGraph object at 0x000001EA194F1EB8>, <reworked_graph.ReworkedGraph object at 0x000001EA194F1F98>, <reworked_graph.ReworkedGraph object at 0x000001EA19504B00>]
new system 2
[<reworked_graph.ReworkedGraph object at 0x000001EA19504588>, <reworked_graph.ReworkedGraph object at 0x000001EA195042B0>, <reworked_graph.ReworkedGraph object at 0x000001EA195040F0>, <reworked_graph.ReworkedGraph object at 0x000001EA195044E0>, <reworked_graph.ReworkedGraph object at 0x000001EA19504748>]
new system 3
[<reworked_

[<reworked_graph.ReworkedGraph object at 0x000001EA196DD6D8>, <reworked_graph.ReworkedGraph object at 0x000001EA196DDC88>, <reworked_graph.ReworkedGraph object at 0x000001EA18DEFC18>, <reworked_graph.ReworkedGraph object at 0x000001EA18E0DE48>, <reworked_graph.ReworkedGraph object at 0x000001EA197E7FD0>]
new system 44
[<reworked_graph.ReworkedGraph object at 0x000001EA18E184A8>, <reworked_graph.ReworkedGraph object at 0x000001EA190D2BA8>, <reworked_graph.ReworkedGraph object at 0x000001EA190BCC50>, <reworked_graph.ReworkedGraph object at 0x000001EA1917DD30>, <reworked_graph.ReworkedGraph object at 0x000001EA1917DCF8>]
new system 45
[<reworked_graph.ReworkedGraph object at 0x000001EA1917DD68>, <reworked_graph.ReworkedGraph object at 0x000001EA1917DDD8>, <reworked_graph.ReworkedGraph object at 0x000001EA19544F98>, <reworked_graph.ReworkedGraph object at 0x000001EA190D2BE0>, <reworked_graph.ReworkedGraph object at 0x000001EA1968DA90>]
new system 46
[<reworked_graph.ReworkedGraph object at

[<reworked_graph.ReworkedGraph object at 0x000001EA192ECA20>, <reworked_graph.ReworkedGraph object at 0x000001EA192EC9B0>, <reworked_graph.ReworkedGraph object at 0x000001EA192F2A90>, <reworked_graph.ReworkedGraph object at 0x000001EA192ECD68>, <reworked_graph.ReworkedGraph object at 0x000001EA192F5128>]
new system 10
[<reworked_graph.ReworkedGraph object at 0x000001EA192F6470>, <reworked_graph.ReworkedGraph object at 0x000001EA19778D30>, <reworked_graph.ReworkedGraph object at 0x000001EA192EEBE0>, <reworked_graph.ReworkedGraph object at 0x000001EA192F90F0>, <reworked_graph.ReworkedGraph object at 0x000001EA192FE588>]
new system 11
[<reworked_graph.ReworkedGraph object at 0x000001EA193016D8>, <reworked_graph.ReworkedGraph object at 0x000001EA19301710>, <reworked_graph.ReworkedGraph object at 0x000001EA19309C88>, <reworked_graph.ReworkedGraph object at 0x000001EA19306FD0>, <reworked_graph.ReworkedGraph object at 0x000001EA193102B0>]
new system 12
[<reworked_graph.ReworkedGraph object at

[<reworked_graph.ReworkedGraph object at 0x000001EA19971C88>, <reworked_graph.ReworkedGraph object at 0x000001EA1996BC50>, <reworked_graph.ReworkedGraph object at 0x000001EA1997C1D0>, <reworked_graph.ReworkedGraph object at 0x000001EA19988DA0>, <reworked_graph.ReworkedGraph object at 0x000001EA19980668>]
new system 40
[<reworked_graph.ReworkedGraph object at 0x000001EA199905C0>, <reworked_graph.ReworkedGraph object at 0x000001EA19993E10>, <reworked_graph.ReworkedGraph object at 0x000001EA199A2F98>, <reworked_graph.ReworkedGraph object at 0x000001EA199A4390>, <reworked_graph.ReworkedGraph object at 0x000001EA199A4908>]
